# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip show python-dotenv
#!pip show openai

In [ ]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key = "X" #os.environ['OPENAI_API_KEY']
client = OpenAI(api_key='X')
print(client.api_key)

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [ ]:
def get_completion(prompt, model="gpt-4-1106-preview"):
    messages = [{"role":"user", "content":prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    print(response_message)
    return response_message

In [ ]:
def get_completion(prompt, model="gpt-4-1106-preview"):
    #GPT_MODEL = "gpt-4-1106-preview" #"gpt-3.5-turbo-1106"
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    response_message = response.choices[0].message.content
    print(response_message )
    return response_message
    #return response.choices[0].message["content"]


In [ ]:
get_completion("What is 1+1?")

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """Mexican spanish \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)
response = get_completion(prompt)

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#OLD %pip install --upgrade langchain
%pip install -U langchain-openai

### Model

In [ ]:
#OLD from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0,  COLDER = LOWER, Less Creative  WARMER = HIGHER. More Creative
chat = ChatOpenAI(temperature=0.8)
chat

### Prompt template

In [ ]:
style = """Mexican spanish \
in a calm and respectful tone
"""

text = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
print(prompt_template)
#ABOVE gets translated to :
#prompt = PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')
#         ['style', 'text']

#print(prompt_template.messages[0].prompt)
#print(prompt_template.messages[0].prompt.input_variables)

# OTHER EXAMPLES
#from langchain_core.prompts import ChatPromptTemplate
#
#        template = ChatPromptTemplate.from_messages([
#            ("system", "You are a helpful AI bot. Your name is {name}."),
#            ("human", "Hello, how are you doing?"),
#            ("ai", "I'm doing well, thanks!"),
#            ("human", "{user_input}"),
#        ])
#
#        messages = template.format_messages(
#            name="Bob",
#            user_input="What is your name?" )

#from langchain import PromptTemplate
#
#template = """Question: {question}
#
#Let's think step by step.
#
#Answer: """
#
#prompt = PromptTemplate(template=template, input_variables=["question"])
#prompt.format(question="Can Barack Obama have a conversation with George Washington?")

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
def format_messages(messages):
    """
    Formats messages for the language model prompt.
    """
    formatted_messages = []
    for message in messages:
        style = message["style"]
        text = message["text"]
        formatted_messages.append(f"{style}: {text}")
    return "\n".join(formatted_messages)

In [ ]:
conversation = [
        {"style": customer_style, "text": customer_email}
    ]

In [ ]:
print(format_messages(conversation))

In [ ]:
#customer_messages = prompt_template.format_messages(
#                    style=customer_style,
#                    text=customer_email)
#print(customer_messages)


bookTemplate_string = """Create a summary of the book \
passed by triple backticks. \
text: ```{name} {place}```
"""
prompt_bookTemplate = ChatPromptTemplate.from_template(bookTemplate_string)
print(prompt_bookTemplate.messages)
# Calling the method with keyword arguments to replace placeholders
#   keyword arguments to use for filling in template variables
#   in all the template messages in this chat template.
styled_book_message = prompt_bookTemplate.format_messages(name="Alice", place="In Wonderland")
print(styled_book_message)
styled_book_message[0].content



In [ ]:
#print(type(customer_messages))
#print(type(customer_messages[0]))
#print(customer_messages[0])

msgs= format_messages(conversation)

response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # Choose the appropriate engine for your needs
    messages=[{"role": "system", "content": "You are an assistant that remembers users."}],
    max_tokens=150,  # Adjust based on how long you expect the response to be
    temperature=0.8,  # Adjust to control the randomness of the response
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

#print(response.choices)
print(response.choices[0].finish_reason)
print(response.choices[0].message)

#USING A STYLE
response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # Choose the appropriate engine for your needs
    messages=[
        {"role": "system", "content": "You are an AI trained to assist with customer service inquiries."},
        {"role": "user", "content": styled_book_message[0].content}
    ],
    max_tokens=150,  # Adjust based on how long you expect the response to be
    temperature=0.8,  # Adjust to control the randomness of the response
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response.choices[0].finish_reason)
print(response.choices[0].message)

#
#                    "messages": messages,
#                    "model": model,
#                    "frequency_penalty": frequency_penalty,
#                    "function_call": function_call,
#                    "functions": functions,
#                    "logit_bias": logit_bias,
#                    "logprobs": logprobs,
#                    "max_tokens": max_tokens,
#                    "n": n,
#                    "presence_penalty": presence_penalty,
#                    "response_format": response_format,
#                    "seed": seed,
#                    "stop": stop,
#                    "stream": stream,
#                    "temperature": temperature,
#                    "tool_choice": tool_choice,
#                    "tools": tools,
#                    "top_logprobs": top_logprobs,
#                    "top_p": top_p,
#                    "user": user,
#

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # Choose the appropriate engine for your needs
    messages=[
        {"role": "system", "content": "You are an AI trained to assist with customer service inquiries."},
        {"role": "user", "content": service_messages[0].content}
    ])

print(service_response.choices[0].finish_reason)
print(service_response.choices[0].message)

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [ ]:
from langchain.prompts import ChatPromptTemplate

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

review_message = prompt_template.format_messages(text=customer_review)
print(review_message[0].content)

review_response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # Choose the appropriate engine for your needs
    messages=[
        {"role": "system", "content": "You are an AI trained to assist with customer service inquiries."},
        {"role": "user", "content": review_message[0].content}
    ])

print(review_response.choices[0].finish_reason)
print(review_response.choices[0].message)

# You will get an error by running this line of code 
# because'gift' is not a dictionary
#'gift' is a string
content = review_response.choices[0].message.dict
print(content)

### Parse the LLM output string into a Python dictionary

In [154]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [155]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [156]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [157]:
format_instructions = output_parser.get_format_instructions()


In [158]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [159]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [160]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [161]:
review_response = client.chat.completions.create(
    model="gpt-4-1106-preview",  # Choose the appropriate engine for your needs
    messages=[
        {"role": "system", "content": "You are an AI trained to assist with customer service inquiries."},
        {"role": "user", "content": messages[0].content}
    ])

print(review_response.choices[0].finish_reason)
print(review_response.choices[0].message)

stop
ChatCompletionMessage(content='```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```', role='assistant', function_call=None, tool_calls=None)


In [162]:
print(review_response.choices[0].message.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [163]:
output_dict = output_parser.parse(review_response.choices[0].message.content)

In [164]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [165]:
type(output_dict)

dict

In [169]:
print(output_dict.get('delivery_days'))
print(output_dict.get('gift'))
print(output_dict.get('price_value'))

2
True
["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
